# Historical analysis for `NSE`

## `TO BE CLEANED UP`

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

1

# Imports

In [2]:
# --- IMPORTS ---
# ---------------

import asyncio
import glob
import io
import json
import math
import pickle
from dataclasses import dataclass
from datetime import date, datetime, time, timedelta, timezone
from itertools import chain
from pathlib import Path
from typing import List, Union

import numpy as np
import pandas as pd
import pytz
import requests
from bs4 import BeautifulSoup
from from_root import from_root
from ib_async import IB, MarketOrder, Option, Order, util
from loguru import logger
from pandas import json_normalize
from scipy.integrate import quad
from scipy.stats import norm
from tqdm import tqdm
from utils import load_config

# Constants, Declarations

In [3]:
# --- CONSTANTS ---
# -----------------

config = load_config()
PORT = port = config.get("PORT")

PUTSTDMULT = 1.8
CALLSTDMULT = 2.2


# Symbol Maps
IDXHISTSYMMAP = config.get("IDXHISTSYMMAP")

ROOT = from_root()

# Functions

In [58]:
# --- MAIN FUNCTION ---
# ----------------------


def all_early_fnos(fons: Union[List, set], save: bool = False) -> pd.DataFrame:
    """Make all early fnos"""

    timer = Timer("Making earliest nakeds")
    timer.start()

    dfs = []

    for symbol in fnos:

        try:
            df_nakeds = make_earliest_naked_opts(symbol)
            df_nakeds = df_nakeds[df_nakeds.xPrice > 0]
            
        except AttributeError as e:
            logger.error(e)
            df_nakeds = None

        dfs.append(df_nakeds)

        # collect dfs and save
        if dfs:
            df = pd.concat(dfs, axis=0, ignore_index=True)

            if save:
                pickle_me(df, ROOT / "data" / "earliest_nakeds.pkl")
        else:
            df = dfs

    timer.stop()

    return df


# --- UTILITIES ---
# ----------------------


def get_files_from_patterns(pattern: str = "/*nakeds*") -> list:
    """Gets list of files from a folder matching pattern given"""

    ROOT = from_root()

    return glob.glob(f"{ROOT / 'data'}{pattern}")


def nse_ban_list() -> list:
    """Gets scrips banned today"""

    url = "https://nsearchives.nseindia.com/content/fo/fo_secban.csv"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 "
        "(KHTML, like Gecko) "
        "Chrome/80.0.3987.149 Safari/537.36",
        "accept-language": "en,gu;q=0.9,hi;q=0.8",
        "accept-encoding": "gzip, deflate, br",
    }

    base_url = "https://www.nseindia.com"

    with requests.Session() as session:
        request = session.get(base_url, headers=headers, timeout=5)
        cookies = dict(request.cookies)
        response = session.get(url, headers=headers, timeout=5, cookies=cookies)

    df = pd.read_csv(io.StringIO(response.text))
    ban_list = df.iloc[:, 0].tolist()

    return ban_list


def live_cache(app_name):
    """Caches the output for time_out specified. This is done in order to
    prevent hitting live quote requests to NSE too frequently. This wrapper
    will fetch the quote/live result first time and return the same result for
    any calls within 'time_out' seconds.

    Logic:
        key = concat of args
        try:
            cached_value = self._cache[key]
            if now - self._cache['tstamp'] < time_out
                return cached_value['value']
        except AttributeError: # _cache attribute has not been created yet
            self._cache = {}
        finally:
            val = fetch-new-value
            new_value = {'tstamp': now, 'value': val}
            self._cache[key] = new_value
            return val

    """

    def wrapper(self, *args, **kwargs):
        """Wrapper function which calls the function only after the timeout,
        otherwise returns value from the cache.

        """
        # Get key by just concating the list of args and kwargs values and hope
        # that it does not break the code :P
        inputs = [str(a) for a in args] + [str(kwargs[k]) for k in kwargs]
        key = app_name.__name__ + "-".join(inputs)
        now = datetime.now()
        time_out = self.time_out
        try:
            cache_obj = self._cache[key]
            if now - cache_obj["timestamp"] < timedelta(seconds=time_out):
                return cache_obj["value"]
        except:
            self._cache = {}
        value = app_name(self, *args, **kwargs)
        self._cache[key] = {"value": value, "timestamp": now}
        return value

    return wrapper


def split_dates(days: int = 365, chunks: int = 50) -> list:
    """splits dates into buckets, based on chunks"""

    end = datetime.today()
    periods = int(days / chunks)
    start = end - timedelta(days=days)

    if days < chunks:
        date_ranges = [(start, end)]
    else:
        dates = pd.date_range(start, end, periods).date
        date_ranges = list(
            zip(pd.Series(dates), pd.Series(dates).shift(-1) + timedelta(days=-1))
        )[:-1]

    # remove last tuple having period as NaT
    if any(pd.isna(e) for element in date_ranges for e in element):
        date_ranges = date_ranges[:-1]

    return date_ranges


def make_date_range_for_stock_history(
    symbol: str, days: int = 365, chunks: int = 50
) -> list:
    """Uses `split_dates` to make date range for stock history"""

    date_ranges = split_dates(days=days, chunks=chunks)

    series = "EQ"

    ranges = [
        {
            "symbol": symbol,
            "from": start.strftime("%d-%m-%Y"),
            "to": end.strftime("%d-%m-%Y"),
            "series": f'["{series}"]',
        }
        for start, end in date_ranges
    ]

    return ranges


def clean_stock_history(result: list) -> pd.DataFrame:
    """Cleans output of"""

    df = pd.concat(
        [pd.DataFrame(r.get("data")) for r in result], axis=0, ignore_index=True
    )

    # ...clean columns

    mapping = {
        "CH_SYMBOL": "nse_symbol",
        "TIMESTAMP": "date",
        "CH_OPENING_PRICE": "open",
        "CH_TRADE_HIGH_PRICE": "high",
        "CH_TRADE_LOW_PRICE": "low",
        "CH_CLOSING_PRICE": "close",
        "CH_TOT_TRADED_QTY": "qty_traded",
        "CH_TOT_TRADED_VAL": "value_traded",
        "CH_TOTAL_TRADES": "trades",
        "VWAP": "vwap",
        "updatedAt": "extracted_on",
    }

    df = df[[col for col in mapping.keys() if col in df.columns]].rename(
        columns=mapping
    )

    # ...convert column datatypes

    astype_map = {
        **{
            k: "float"
            for k in ["open", "high", "low", "close", "value_traded", "trades", "vwap"]
        },
        **{"qty_traded": "int"},
    }

    df = df.astype(astype_map)

    # ...change date columns to utc

    replace_cols = ["date", "extracted_on"]
    df1 = df[replace_cols].map(lambda x: datetime.fromisoformat(x))
    df = df.assign(date=df1.date, extracted_on=df1.extracted_on)

    return df


def clean_index_history(results: list) -> pd.DataFrame:
    """cleans index history and builds it as a dataframe"""

    df = pd.concat(
        [pd.DataFrame(json.loads(r.get("d"))) for r in results], ignore_index=True
    )

    # clean the df

    # ...drop unnecessary columns

    df = df.drop(df.columns[[0, 1]], axis=1)

    # ...rename
    df.columns = ["nse_symbol", "date", "open", "high", "low", "close"]

    # ...convert nse_symbol to IB's symbol
    df = pd.concat(
        [
            df.nse_symbol.map(
                {"Nifty Bank": "BANKNIFTY", "Nifty 50": "NIFTY50"}
            ).rename("symbol"),
            df,
        ],
        axis=1,
    )

    utc_dates = df.date.apply(lambda x: convert_to_utc_datetime(x, eod=True))

    df = df.assign(date=utc_dates)

    # .....convert ohlc to numeric
    convert_dict = {k: "float" for k in ["open", "high", "low", "close"]}

    df = df.astype(convert_dict)

    # .....sort by date
    df.sort_values(["nse_symbol", "date"], inplace=True, ignore_index=True)

    # .....add extract_date
    now = datetime.now()
    utc_now = now.astimezone(timezone.utc)
    df = df.assign(extracted_on=utc_now)

    return df


def clean_ib_util_df(contracts: Union[list, pd.Series]) -> pd.DataFrame:
    """Cleans ib_async's util.df to keep only relevant columns"""

    df1 = pd.DataFrame([])  # initialize

    if isinstance(contracts, list):
        df1 = util.df(contracts)
    elif isinstance(contracts, pd.Series):
        try:
            contract_list = list(contracts)
            df1 = util.df(contract_list)  # it could be a series
        except (AttributeError, ValueError):
            logger.error(f"cannot clean type: {type(contracts)}")
    else:
        logger.error(f"cannot clean unknowntype: {type(contracts)}")

    if not df1.empty:

        df1.rename(
            {"lastTradeDateOrContractMonth": "expiry"}, axis="columns", inplace=True
        )

        df1 = df1.assign(expiry=pd.to_datetime(df1.expiry))
        cols = list(df1.columns[:6])
        cols.append("multiplier")
        df2 = df1[cols]
        df2 = df2.assign(contract=contracts)

    else:
        df2 = None

    return df2


def nse2ib(nse_list):
    """Converts nse to ib friendly symbols"""

    subs = {"M&M": "MM", "M&MFIN": "MMFIN", "L&TFH": "LTFH", "NIFTY": "NIFTY50"}

    list_without_percent_sign = list(map(subs.get, nse_list, nse_list))

    # fix length to 9 characters
    ib_equity_fnos = [s[:9] for s in list_without_percent_sign]

    return ib_equity_fnos


def convert_to_utc_datetime(date_string, eod=False):
    """Converts nse date strings to utc datetimes. If eod is chosen 3:30 PM IST is taken."""

    # List of possible date formats
    date_formats = ["%d-%b-%Y", "%d %b %Y", "%Y-%m-%d %H:%M:%S.%f%z"]

    for date_format in date_formats:
        try:
            dt = datetime.strptime(date_string, date_format)

            # If the parsed datetime doesn't have timezone info, assume it's UTC
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=pytz.UTC)
            else:
                # If it has timezone info, convert to UTC
                dt = dt.astimezone(pytz.UTC)

            if eod:
                # Set time to 3:30 PM India time for all formats when eod is True
                india_time = time(hour=15, minute=30)
                india_tz = pytz.timezone("Asia/Kolkata")
                dt = india_tz.localize(datetime.combine(dt.date(), india_time))
                dt = dt.astimezone(pytz.UTC)
            elif dt.time() == time(0, 0):  # If time is midnight (00:00:00)
                # Keep it as midnight UTC
                dt = dt.replace(hour=0, minute=0, second=0, microsecond=0)

            return dt
        except ValueError:
            continue

    # If none of the formats work, raise an error
    raise ValueError(f"Unable to parse date string: {date_string}")


def convert_to_numeric(col: pd.Series):
    """convert to numeric if possible, only for object dtypes"""

    if col.dtype == "object":
        try:
            return pd.to_numeric(col)
        except ValueError:
            return col
    return col


def convert_daily_volatility_to_yearly(daily_volatility, days: float = 252):
    return daily_volatility * math.sqrt(days)


def equity_iv_df(quotes: dict) -> pd.DataFrame:
    """Build a core df with symbol, undPrice, expiry, strike, volatilities, lot and price."""

    flat_data = json_normalize(quotes, sep="-")

    # get symbol, lot and underlying pricefrom quote

    symbol = quotes.get("info").get("symbol")

    lot = (
        quotes["stocks"][0].get("marketDeptOrderBook").get("tradeInfo").get("marketLot")
    )

    undPrice = quotes["underlyingValue"]

    # build the df
    df = pd.DataFrame(flat_data)

    df = pd.DataFrame(
        [
            {
                "nse_symbol": symbol,
                "ib_symbol": symbol,
                "instrument": quotes.get("stocks")[i]
                .get("metadata")
                .get("instrumentType"),
                "expiry": quotes.get("stocks")[i].get("metadata").get("expiryDate"),
                "undPrice": undPrice,
                "safe_strike": 0,
                "right": quotes.get("stocks")[i].get("metadata").get("optionType")[:1],
                "strike": quotes.get("stocks")[i].get("metadata").get("strikePrice"),
                "dte": np.nan,
                "hv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("annualisedVolatility"),
                "iv": quotes.get("stocks")[i]
                .get("marketDeptOrderBook")
                .get("otherInfo")
                .get("impliedVolatility"),
                "lot": lot,
                "price": quotes.get("stocks")[i].get("metadata").get("lastPrice"),
            }
            for i in range(len(quotes))
        ]
    )

    # Convert nse_symbol to symbol
    df = df.assign(ib_symbol=nse2ib(df.nse_symbol))

    # Convert expiry to UTC NSE eod
    df = df.assign(
        expiry=df.expiry.apply(lambda x: convert_to_utc_datetime(x, eod=True))
    )

    df = df.assign(dte=get_dte(df.expiry))

    # Convert the rest to numeric
    df = df.apply(convert_to_numeric)

    # Convert to %ge
    df.iv = df.iv / 100
    df.hv = df.hv / 100

    # Change instrument type
    instrument_dict = {
        "Stock": "STK",
        "Options": "OPT",
        "Currency": "FX",
        "Index": "IDX",
        "Futures": "FUT",
    }

    inst = df.instrument.str.split()

    s = inst.apply(lambda x: "".join(instrument_dict[item] for item in x))

    df = df.assign(instrument=s)

    df = df[df.instrument.isin(["IDXOPT", "STKOPT"])]

    return df


def find_closest_strike(df, above=False):
    """
    Finds the row with the strike closest to the undPrice.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    above (bool): If True, find the closest strike above undPrice. If False, find the closest strike below undPrice.

    Returns:
    pd.DataFrame: A DataFrame with the single row that has the closest strike.
    """
    undPrice = df["undPrice"].iloc[0]  # Get the undPrice from the first row

    if above:
        # Filter for strikes above undPrice
        mask = df["strike"] > undPrice
    else:
        # Filter for strikes below undPrice
        mask = df["strike"] < undPrice

    if not mask.any():
        return pd.DataFrame()  # Return an empty DataFrame if no rows match the criteria

    # Calculate the absolute difference between strike and undPrice
    diff = np.abs(df.loc[mask, "strike"] - undPrice)

    # Find the index of the row with the minimum difference
    closest_index = diff.idxmin()

    # Return the closest row as a DataFrame
    return df.loc[[closest_index]]


def get_dte(s: pd.Series) -> pd.Series:
    """Gets days to expiry. Expects series of UTC timestamps"""

    now_utc = datetime.now(pytz.UTC)
    return (s - now_utc).dt.total_seconds() / (24 * 60 * 60)


def fbfillnas(ser: pd.Series) -> pd.Series:
    """Fills nan in series forwards first and then backwards"""

    s = ser.copy()

    # Find the first non-NaN value
    first_non_nan = s.dropna().iloc[0]

    # Fill first NaN with the first non-NaN value
    s.iloc[0] = first_non_nan

    # Fill remaining NaN values with the next valid value
    s = s.fillna(s.bfill())

    # Fill remaining NaN values with the previous valid value
    s = s.fillna(s.ffill())

    return ser.fillna(s)


def get_a_stdev(iv: float, price: float, dte: float) -> float:
    """Gives 1 Standard Deviation value for annual iv"""

    return iv * price * math.sqrt(dte / 365)


def get_prob(sd):
    """Compute probability of a normal standard deviation

    Arg:
        (sd) as standard deviation
    Returns:
        probability as a float

    """
    prob = quad(lambda x: np.exp(-(x**2) / 2) / np.sqrt(2 * np.pi), -sd, sd)[0]
    return prob


def get_prec(v: float, base: float) -> float:
    """Gives the precision value

    Args:
       (v) as value needing precision in float
       (base) as the base value e.g. 0.05
    Returns:
        the precise value"""

    try:
        output = round(round((v) / base) * base, -int(math.floor(math.log10(base))))
    except Exception:
        output = None

    return output


# Prettify columns to show based on a dictionary map
def pretty_columns(df: pd.DataFrame, col_map: dict) -> list:
    """prettifies columns based on column map dictionary"""

    cols = [v for _, v in col_map.items() if v in df.columns]
    return cols


def get_ib_margin(contract: Option, order: MarketOrder) -> dict:
    """Gets margin and commission of a contract"""

    with IB().connect(port=port) as ib:
        if contract.conId == 0:  # qualify raw contracts
            contract = next(iter(ib.qualifyContracts(contract)))
        wif = ib.whatIfOrder(contract, order)

    # margin = float(wif.initMarginChange) # initial margin is too high compared to Zerodha, SAMCO
    margin = float(wif.maintMarginChange)
    comm = min(wif.commission, wif.minCommission, wif.maxCommission)

    return {"contract": contract, "margin": margin, "comm": comm}


def make_contracts_orders(
    df: pd.DataFrame, EXCHANGE: str = "NSE", action: str = "SELL"
) -> pd.DataFrame:
    """Makes df[contract, order] from option df. Used for margin check"""

    df_out = pd.DataFrame(
        {
            "contract": df.apply(
                lambda row: Option(
                    row.ib_symbol,
                    # util.formatIBDatetime(row.expiry.date())[:8],
                    util.formatIBDatetime(row.expiry.date()),
                    row.strike,
                    row.right,
                    EXCHANGE,
                ),
                axis=1,
            ),
            "order": df.apply(
                lambda row: MarketOrder(action=action, totalQuantity=row.lot), axis=1
            ),
        }
    )

    return df_out


def get_ib_margin_comms(df: pd.DataFrame) -> pd.DataFrame:
    """Qualified Contracts, Margins and Commissions from an options df"""

    symbol = df.ib_symbol.iloc[0]
    df_cos = make_contracts_orders(df)

    cts = [d if d.conId == 0 else None for d in df_cos.contract]
    with IB().connect(port=port) as ib:
        if len(cts) > 40:
            ib.qualifyContracts(*tqdm(cts, desc=f"Qualifying {symbol} options"))
        else:
            ib.qualifyContracts(*cts)
            
        df_cos.contract = cts
        ib.disconnect()

    if len(df_cos) > 1:  # use tqdm.pandas.progress_apply()
        tqdm.pandas(desc=f"Calculating {symbol} margins")
        data = df_cos.progress_apply(
            lambda row: get_ib_margin(row.contract, row.order), axis=1
        )
    else:
        data = df_cos.apply(lambda row: get_ib_margin(row.contract, row.order), axis=1)

    df_mcom = pd.DataFrame.from_dict(data.to_dict()).T

    # replace raw contracts with qualified
    df_q = df_cos.join(df_mcom, how="outer", lsuffix="_left").drop(
        ["contract_left", "order"], axis=1
    )

    # merge margins and commissions
    df_opts = df.merge(df_q, left_index=True, right_index=True)

    # rename instrument to secType for IB
    df_opts = df_opts.assign(
        instrument=df_opts.contract.apply(lambda s: s.secType)
    ).rename(columns={"instrument": "secType"}, errors="ignore")

    return df_opts


def black_scholes(
    S: float,  # underlying
    K: float,  # strike
    T: float,  # years-to-expiry
    r: float,  # risk-free rate
    sigma: float,  # implied volatility
    option_type: str,  # Put or Call right
) -> float:
    """Black-Scholes Option Pricing Model"""

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "C":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "P":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'C' for Call and 'P' for Put.")

    return price


def make_earliest_naked_opts(
    symbol: str,  # nse_symbol. can be equity or index
) -> pd.DataFrame:
    """Make target options for nakeds with earliest expiry"""

    # Instantiate nse
    nse = NSEfnos()

    # Get the basic quote
    fno_quote = nse.stock_quote_fno(symbol)

    # make the fno df with iv, hv, lot
    df_fno = equity_iv_df(fno_quote)

    # get margins and commissions from ib
    df_mcom = get_ib_margin_comms(df_fno)

    # Remove zero IVs
    df_mcom = df_mcom[df_mcom.iv > 0]

    # Get the risk free rate
    rbi = RBI()
    risk_free_rate = rbi.repo_rate() / 100

    # Compute the expected price from black_scholes
    bsPrice = df_mcom.apply(
        lambda row: black_scholes(
            S=row["undPrice"],
            K=row["strike"],
            T=row["dte"] / 365,  # Convert days to years
            r=risk_free_rate,
            sigma=row["iv"],
            option_type=row["right"],
        ),
        axis=1,
    )

    # Adjust the expected price precision. Ensure 0.05 as the minimum price.
    df = df_mcom.assign(bsPrice=bsPrice.apply(lambda x: get_prec(x, base=0.05)))

    # get the safe-strike, based on std multiples
    df_sp = pd.concat(
        [
            df.right,
            pd.Series(
                df.iv * df.undPrice * (df.dte / 365).apply(math.sqrt), name="sdev"
            ),
            df.undPrice,
        ],
        axis=1,
    )

    safe_strike = np.where(
        df_sp.right == "P",
        (df_sp.undPrice - df_sp.sdev * PUTSTDMULT).astype("int"),
        (df_sp.undPrice + df_sp.sdev * CALLSTDMULT).astype("int"),
    )

    df = df.assign(safe_strike=safe_strike)

    # derive expected price from safe_strike
    df = df.assign(
        xPrice=abs(df.safe_strike - df.strike).apply(lambda x: max(get_prec(x, 0.05), 0.05))
    )

    # calculate the return on margin (rom)
    df = df.assign(rom=df.xPrice * df.lot / df.margin * 365 / df.dte)

    # Sort by likeliest
    df_out = df.loc[(df.xPrice / df.price).sort_values().index]

    return df_out.reset_index(drop=True)

def get_fnos_from_naked_pickles(maxmargin: int=300000):
    """Makes fnos from existing naked pickles"""

    files = get_files_from_patterns("/*nakeds*")

    try:
        df = pd.concat(
            [get_pickle(f) for f in files],
            ignore_index=True,
        )
    
        # Sort by safe_strike: strike ratio to get the safest calls and puts
    
        # ... group calls
        gc = (
            df[df.right == "C"]
            .assign(ratio=df.safe_strike / df.strike)
            .sort_values("ratio")
            .groupby("ib_symbol")
        )
        df_calls = gc.head(2).sort_values(["ib_symbol", "strike"], ascending=[True, False])
        dfc = df_calls[df_calls.margin < maxmargin]
        dfc = dfc.assign(ratio=dfc.safe_strike / dfc.strike).sort_values("ratio")
    
        # ... group puts
        gc = (
            df[df.right == "P"]
            .assign(ratio=df.strike / df.safe_strike)
            .sort_values("ratio")
            .groupby("ib_symbol")
        )
        df_puts = gc.head(2).sort_values(["ib_symbol", "strike"], ascending=[True, False])
        dfp = df_puts[df_puts.margin < maxmargin]
        dfp = dfp.assign(ratio=dfp.strike / dfp.safe_strike).sort_values("ratio")
    
        # ... prepare the nakeds to order
        df_nakeds = pd.concat([dfc, dfp], axis=0, ignore_index=True)

        # ... remove dubious xPrice which are less than option price
        df_nakeds = df_nakeds[df_nakeds.xPrice > df_nakeds.price]

        df_nakeds = df_nakeds.reset_index(drop=True)
        
    except ValueError:
        df_nakeds = None

    return df_nakeds


# --- PICKLE UTILITIES ----
# -------------------------


def pickle_me(obj, file_name_with_path: Path):
    """Pickles objects in a given path"""

    with open(str(file_name_with_path), "wb") as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)


def get_pickle(path: Path, print_msg: bool = True):
    """Gets pickled object"""

    output = None  # initialize

    try:
        with open(path, "rb") as f:
            output = pickle.load(f)
    except FileNotFoundError:
        if print_msg:
            logger.error(f"file not found: {path}")

    return output


# --- ORDER HANDLING ---
# -----------------------


def place_orders(ib: IB, cos: Union[tuple, list], blk_size: int = 25) -> List:
    """!!!CAUTION!!!: This places orders in the system
    ---
    NOTE: cos could be a single (contract, order)
          or a tuple/list of ((c1, o1), (c2, o2)...)
          made using tuple(zip(cts, ords))
    ---
    USAGE:
    ---
    cos = tuple((c, o) for c, o in zip(contracts, orders))
    with IB().connect(port=port) as ib:
        ordered = place_orders(ib=ib, cos=cos)
    """

    trades = []

    if isinstance(cos, (tuple, list)) and (len(cos) == 2):
        c, o = cos
        trades.append(ib.placeOrder(c, o))

    else:
        cobs = {cos[i : i + blk_size] for i in range(0, len(cos), blk_size)}

        for b in tqdm(cobs):
            for c, o in b:
                td = ib.placeOrder(c, o)
                trades.append(td)
            ib.sleep(0.75)

    return trades


def get_open_orders(ib, is_active:bool=False) -> pd.DataFrame:
    """Gets open orders - blocking version"""

    ACTIVE_STATUS = "ApiPending, PendingSubmit, PreSubmitted, Submitted".split(",")

    df_openords = OpenOrder().empty()  # Initialize open orders

    trades = ib.reqAllOpenOrders()
    # trades = ib.trades()
    # ib.sleep(1) # time to take in all open orders

    if trades:

        all_trades_df = (
            clean_ib_util_df([t.contract for t in trades])
            .join(util.df(t.orderStatus for t in trades))
            .join(util.df(t.order for t in trades), lsuffix="_")
        )

        order = pd.Series([t.order for t in trades], name="order")

        all_trades_df = all_trades_df.assign(order=order)

        all_trades_df.rename(
            {"lastTradeDateOrContractMonth": "expiry"}, axis="columns", inplace=True
        )

        # all_trades_df = all_trades_df[all_trades_df.status.isin(ACTIVE_STATUS)]

        trades_cols = df_openords.columns

        dfo = all_trades_df[trades_cols]
        
        if is_active:
            dfo = dfo[dfo.status.isin(ACTIVE_STATUS)]

        # dfo = dfo.assign(expiry=pd.to_datetime(dfo.expiry))
    
    return dfo


def quick_pf(ib) -> Union[None, pd.DataFrame]:
    """Gets the portfolio dataframe"""
    pf = ib.portfolio()  # returns an empty [] if there is nothing in the portfolio

    if pf != []:
        df_pf = util.df(pf)
        df_pf = (util.df(list(df_pf.contract)).iloc[:, :6]).join(
            df_pf.drop(columns=["account"])
        )
        df_pf = df_pf.rename(
            columns={
                "lastTradeDateOrContractMonth": "expiry",
                "marketPrice": "mktPrice",
                "marketValue": "mktVal",
                "averageCost": "avgCost",
                "unrealizedPNL": "unPnL",
                "realizedPNL": "rePnL",
            }
        )
    else:
        df_pf = Portfolio().empty()

    return df_pf


def cancel_all():
    """Cancels all orders"""

    with IB().connect(port=port, clientId=10) as ib:
        ib.reqGlobalCancel()


# --- TEST BED ---
# ----------------

def cancel_open_orders(ib) -> pd.DataFrame:
    """!!!Not working!!! --- CHECK"""

    trades = ib.reqAllOpenOrders()  # To kickstart collection of open orders
    ib.sleep(0.3)
    trades = ib.trades()  # Get the trades

    orders = {
        t.order
        for t in trades
        if t.orderStatus.status == "Submitted"
        if t.order.action == "SELL"
    }

    # df_open_orders = get_open_orders(ib)
    # ords = df_open_orders.order.to_list()

    BLK = 25
    ords = list(orders)
    o_blk = [ords[i : i + BLK] for i in range(0, len(ords), BLK)]

    cancels = []

    for ob in o_blk:
        cancels.append([ib.cancelOrder(o) for o in ob])
        ib.sleep(0.3)

    return cancels

# Classes

In [5]:
# --- CLASSES ---
# ---------------


class NSEfnos:
    """Class for All NSE FNOS, including Indexes"""

    time_out = 5
    base_url = "https://www.nseindia.com/api"
    page_url = "https://www.nseindia.com/get-quotes/equity?symbol=LT"
    _routes = {
        "stock_meta": "/equity-meta-info",
        "stock_quote": "/quote-equity",
        "stock_derivative_quote": "/quote-derivative",
        "market_status": "/marketStatus",
        "chart_data": "/chart-databyindex",
        "market_turnover": "/market-turnover",
        "equity_derivative_turnover": "/equity-stock",
        "all_indices": "/allIndices",
        "live_index": "/equity-stockIndices",
        "index_option_chain": "/option-chain-indices",
        "equity_option_chain": "/option-chain-equities",
        "currency_option_chain": "/option-chain-currency",
        "pre_open_market": "/market-data-pre-open",
        "holiday_list": "/holiday-master?type=trading",
        "stock_history": "/historical/cm/equity",  # added by rkv
    }

    def __init__(self):
        self.s = requests.Session()
        h = {
            "Host": "www.nseindia.com",
            "Referer": "https://www.nseindia.com/get-quotes/equity?symbol=SBIN",
            "X-Requested-With": "XMLHttpRequest",
            "pragma": "no-cache",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36",
            "Accept": "*/*",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
        }
        self.s.headers.update(h)
        self.s.get(self.page_url)

    def get(self, route, payload={}):
        url = self.base_url + self._routes[route]
        r = self.s.get(url, params=payload)
        return r.json()

    @live_cache
    def stock_quote(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_quote", data)

    @live_cache
    def stock_quote_fno(self, symbol):
        data = {"symbol": symbol}
        return self.get("stock_derivative_quote", data)

    @live_cache
    def trade_info(self, symbol):
        data = {"symbol": symbol, "section": "trade_info"}
        return self.get("stock_quote", data)

    @live_cache
    def market_status(self):
        return self.get("market_status", {})

    @live_cache
    def chart_data(self, symbol, indices=False):
        data = {"index": symbol + "EQN"}
        if indices:
            data["index"] = symbol
            data["indices"] = "true"
        return self.get("chart_data", data)

    @live_cache
    def tick_data(self, symbol, indices=False):
        return self.chart_data(symbol, indices)

    @live_cache
    def market_turnover(self):
        return self.get("market_turnover")

    @live_cache
    def eq_derivative_turnover(self, type="allcontracts"):
        data = {"index": type}
        return self.get("equity_derivative_turnover", data)

    @live_cache
    def all_indices(self):
        return self.get("all_indices")

    def live_index(self, symbol="NIFTY 50"):
        data = {"index": symbol}
        return self.get("live_index", data)

    @live_cache
    def index_option_chain(self, symbol="NIFTY"):
        data = {"symbol": symbol}
        return self.get("index_option_chain", data)

    @live_cache
    def equities_option_chain(self, symbol):
        data = {"symbol": symbol}
        return self.get("equity_option_chain", data)

    @live_cache
    def currency_option_chain(self, symbol="USDINR"):
        data = {"symbol": symbol}
        return self.get("currency_option_chain", data)

    @live_cache
    def live_fno(self):
        return self.live_index("SECURITIES IN F&O")

    @live_cache
    def pre_open_market(self, key="NIFTY"):
        data = {"key": key}
        return self.get("pre_open_market", data)

    @live_cache
    def holiday_list(self):
        return self.get("holiday_list", {})

    @live_cache
    def stock_history(self, symbol, days: int = 365, chunks: int = 50):

        date_ranges = make_date_range_for_stock_history(symbol, days, chunks)

        result = []
        for dr in date_ranges:
            result.append(self.get("stock_history", dr))

        df = clean_stock_history(result)

        return df

    def equities(self):
        equities_data = nse.live_fno()
        equities = {kv.get("symbol") for kv in equities_data.get("data")}
        return equities

    def indexes(self):
        x = "NIFTY,BANKNIFTY,MIDCPNIFTY,NIFTYNXT50,FINNIFTY"
        return set(x.split(","))


class IDXHistories:

    time_out = 5
    base_url = "https://niftyindices.com"
    idx_symbols = IDX_SYM_HIST_MAP.values()
    url = "https://niftyindices.com/Backpage.aspx/getHistoricaldatatabletoString"

    # prepare `post` header
    post_header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.77 Safari/537.36",
        "Connection": "keep-alive",
        "sec-ch-ua": '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "DNT": "1",
        "X-Requested-With": "XMLHttpRequest",
        "sec-ch-ua-mobile": "?0",
        "Content-Type": "application/json; charset=UTF-8",
        "Origin": "https://niftyindices.com",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Dest": "empty",
        "Referer": "https://niftyindices.com/reports/historical-data",
        "Accept-Language": "en-US,en;q=0.9,hi;q=0.8",
    }

    def __init__(self, days: int = 365) -> None:
        self.s = requests.Session()

        # update session with default headers and get the cookies
        init_header = requests.utils.default_headers()
        init_header.update(
            {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.77 Safari/537.36",
            }
        )
        self.s.headers.update(init_header)
        c = self.s.get(url=self.url)
        self.cookies = c.cookies

    def get(self, payload={}):

        r = self.s.post(
            url=self.url,
            headers=self.post_header,
            cookies=self.cookies,
            data=payload,
            timeout=self.time_out,
        )

        return r.json()

    def make_histories(self, days: int = 365, chunks: int = 50):
        """Makes histories for NIFTY50 and BANKNIFTY, based on number of days provided"""

        date_ranges = split_dates(days=days, chunks=chunks)

        # idx_symbols = ["Nifty Bank", "Nifty 50"]

        # organize the payloads
        payloads = [
            {
                "cinfo": str(
                    {
                        "name": idx_symbol,
                        "startDate": s.strftime("%d-%b-%Y"),
                        "endDate": e.strftime("%d-%b-%Y"),
                        "indexName": idx_symbol,
                    }
                )
            }
            for s, e in date_ranges
            for idx_symbol in self.idx_symbols
        ]
        # get the raw jsons
        results = []

        for payload in tqdm(payloads):
            r = self.get(payload=json.dumps(payload))
            results.append(r)

        df = clean_index_history(results)

        return df


def rbi_tr_to_json(wrapper):
    trs = wrapper.find_all("tr")
    op = {}
    for tr in trs:
        tds = tr.find_all("td")
        if len(tds) >= 2:
            key = tds[0].text.strip()
            val = tds[1].text.replace(":", "").replace("*", "").replace("#", "").strip()

            op[key] = val
    return op


class RBI:
    base_url = "https://www.rbi.org.in/"

    def __init__(self):
        self.s = requests.Session()

    def current_rates(self):
        r = self.s.get(self.base_url)

        bs = BeautifulSoup(r.text, "html.parser")
        wrapper = bs.find("div", {"id": "wrapper"})

        return rbi_tr_to_json(wrapper)

    def repo_rate(self):

        rate = self.current_rates().get("Policy Repo Rate")[:-1]

        return float(rate)


class Timer:
    """Timer providing elapsed time"""

    def __init__(self, name: str = "") -> None:
        self.name = name
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise Exception(f"Timer is running. Use .stop() to stop it")

        now = datetime.now()

        print(f'\n{self.name} started at {now.strftime("%d-%b-%Y %H: %M:%S")}')

        self._start_time = datetime.now()

    def stop(self) -> None:
        if self._start_time is None:
            raise Exception(f"Timer is not running. Use .start() to start it")

        elapsed_time = datetime.now() - self._start_time

        # Extract hours, minutes, seconds from the timedelta object
        hours = elapsed_time.seconds // 3600
        minutes = (elapsed_time.seconds % 3600) // 60
        seconds = elapsed_time.seconds % 60

        print(f"\n...{self.name} took: " + f"{hours:02d}:{minutes:02d}:{seconds:02d}\n")

        self._start_time = None


def empty_the_df(df):
    """Empty the dataclass df"""
    empty_df = pd.DataFrame([df.__dict__]).iloc[0:0]
    return empty_df


@dataclass
class OpenOrder:
    """
    Open order template with Dummy data. Use:\n
    `df = OpenOrder().empty()`
    """

    conId: int = 0
    symbol: str = "Dummy"
    secType: str = "STK"
    expiry: datetime = datetime.now()
    strike: float = 0.0
    right: str = "?"  # Will be 'P' for Put, 'C' for Call
    orderId: int = 0
    order: Order = None
    permId: int = 0
    action: str = "SELL"  # 'BUY' | 'SELL'
    totalQuantity: float = 0.0
    lmtPrice: float = 0.0
    status: str = None

    def empty(self):
        return empty_the_df(self)


@dataclass
class Portfolio:
    """
    Portfolio template with Dummy data. Use:\n
    `df = OpenOrder().empty()`
    """

    conId: int = 0
    symbol: str = "Dummy"
    secType: str = "STK"
    expiry: datetime = datetime.now()
    strike: float = 0.0
    right: str = "?"  # Will be 'P' for Put, 'C' for Call
    position: float = 0.0
    mktPrice: float = 0.0
    mktVal: float = 0.0
    avgCost: float = 0.0
    unPnL: float = 0.0
    rePnL: float = 0.0

    def empty(self):
        return empty_the_df(self)

# BIG TEST

In [17]:
# Initialize FnO class
nse = NSEfnos()

# get executed symbols from pickles
files = get_files_from_patterns("/*nakeds*")
symbols_list = [get_pickle(ROOT / "data" / file).nse_symbol.to_list() for file in files]
executed = set(chain.from_iterable(symbols_list))

# get open order symbols
with IB().connect(port=port, clientId=10) as ib:

    trades = ib.reqAllOpenOrders()
    df_openords = get_open_orders(ib)
    open_orders = set(df_openords.symbol.to_list())

In [18]:
# build the fnos list
fnos = ((set(["BANKNIFTY", "NIFTY"]) | nse.equities()) - executed) - open_orders
fnos = list(fnos - set(nse_ban_list()))
fnos.sort()

len(fnos)

0

In [19]:
# Build the earliest naked options

if fnos:
    df_nakeds = all_early_fnos(fnos, save=True)

In [20]:
df_nakeds = get_fnos_from_naked_pickles()

In [26]:
{"current_#fnos": len(fnos), "unique_symbols": len(df_nakeds.ib_symbol.unique()), "# of orders": len(df_nakeds)}

{'current_#fnos': 0, 'unique_symbols': 156, '# of orders': 576}

In [27]:
df_nakeds.head()

,nse_symbol,ib_symbol,secType,expiry,undPrice,safe_strike,right,strike,dte,hv,iv,lot,price,contract,margin,comm,bsPrice,xPrice,rom,ratio
0,HINDCOPPER,HINDCOPPE,OPT,2024-07-25 10:00:00+00:00,322.40,383,C,400.0,2.390688,0.6474,1.0580,2650,0.15,"Option(conId=699732738, symbol='HINDCOPPE', la...",174790.69,20.0,0.05,17.0,39.35015,0.957500
1,HINDCOPPER,HINDCOPPE,OPT,2024-07-25 10:00:00+00:00,322.40,367,C,380.0,2.390688,0.6474,0.7940,2650,0.10,"Option(conId=699732723, symbol='HINDCOPPE', la...",193314.19,20.0,0.05,13.0,27.207922,0.965789
2,MCX,MCX,OPT,2024-07-25 10:00:00+00:00,3805.00,4372,C,4500.0,2.386690,0.5121,0.8383,200,1.55,"Option(conId=699754888, symbol='MCX', lastTrad...",99279.37,20.0,0.60,128.0,39.434634,0.971556
3,DALBHARAT,DALBHARAT,OPT,2024-07-25 10:00:00+00:00,1773.00,2054,C,2100.0,2.393369,0.3729,0.8911,275,1.05,"Option(conId=707068291, symbol='DALBHARAT', la...",64300.56,20.0,0.45,46.0,30.002604,0.978095
4,SUNTV,SUNTV,OPT,2024-07-25 10:00:00+00:00,779.05,882,C,900.0,2.382097,0.3592,0.7477,1500,0.35,"Option(conId=707129288, symbol='SUNTV', lastTr...",180369.29,20.0,0.15,18.0,22.936889,0.980000


In [28]:
# --- PLACE THE ORDERS !!!

from ib_async import LimitOrder

contracts = df_nakeds.contract.to_list()
orders = [
    LimitOrder(action="SELL", totalQuantity=abs(int(q)), lmtPrice=p)
    for q, p in zip(df_nakeds.lot, df_nakeds.xPrice)
]

In [29]:
cos = tuple((c, o) for c, o in zip(contracts, orders))

with IB().connect(port=3000) as ib:
    ordered = place_orders(ib=ib, cos=cos)

 17%|█████████████▊                                                                     | 4/24 [00:03<00:15,  1.30it/s]Error 201, reqId 61573: Order rejected - reason:Your order has been rejected. The sum of the risk of the outstanding orders likely to execute is greater than <br>the current allowable multiple of excess equity.  Please call the Trade Desk for further detail.
Canceled order: Trade(contract=Option(conId=700290438, symbol='COROMANDE', lastTradeDateOrContractMonth='20240725', strike=1500.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='COROMANDEL24JUL1500PE', tradingClass='COROMANDEL'), order=LimitOrder(orderId=61573, clientId=1, permId=1710300150, action='SELL', totalQuantity=700.0, lmtPrice=36.0, auxPrice=0.0), orderStatus=OrderStatus(orderId=61573, status='Cancelled', filled=0.0, remaining=700.0, avgFillPrice=0.0, permId=1710300150, parentId=0, lastFillPrice=0.0, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=dateti

In [52]:
filename= f"{datetime.now().strftime('%Y%m%d_%Hh%Mm%Ss')}_orders.pkl"
pickle_me(ordered, str(ROOT / "data" / 'zArchive' / filename))

In [59]:
with IB().connect(port=port, clientId=10) as ib:
    df_openords = get_open_orders(ib)

In [60]:
df_openords

,conId,symbol,secType,expiry,strike,right,orderId,order,permId,action,totalQuantity,lmtPrice,status
0,699695193,APOLLOTYR,OPT,2024-07-25,560.0,C,61569,"Order(orderId=61569, clientId=1, permId=171030...",1710300145,SELL,1700.0,24.0,PreSubmitted
1,699721137,EICHERMOT,OPT,2024-07-25,5000.0,C,61568,"Order(orderId=61568, clientId=1, permId=171030...",1710300144,SELL,175.0,213.0,PreSubmitted
2,699737337,IEX,OPT,2024-07-25,185.0,C,61572,"Order(orderId=61572, clientId=1, permId=171030...",1710300147,SELL,3750.0,8.0,PreSubmitted
3,699749833,LTIM,OPT,2024-07-25,5900.0,C,61571,"Order(orderId=61571, clientId=1, permId=171030...",1710300146,SELL,150.0,253.0,PreSubmitted
4,699767074,PETRONET,OPT,2024-07-25,360.0,C,61563,"Order(orderId=61563, clientId=1, permId=171030...",1710300149,SELL,3000.0,15.0,PreSubmitted
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,707104559,UNITDSPR,OPT,2024-07-25,1250.0,P,61476,"Order(orderId=61476, clientId=1, permId=171030...",1710300058,SELL,700.0,57.0,PreSubmitted
94,699745567,JUBLFOOD,OPT,2024-07-25,560.0,P,61482,"Order(orderId=61482, clientId=1, permId=171030...",1710300061,SELL,1250.0,27.0,PreSubmitted
95,699772607,RELIANCE,OPT,2024-07-25,3000.0,P,61481,"Order(orderId=61481, clientId=1, permId=171030...",1710300060,SELL,250.0,144.0,PreSubmitted
96,707084174,HINDUNILV,OPT,2024-07-25,2700.0,P,61485,"Order(orderId=61485, clientId=1, permId=171030...",1710300063,SELL,300.0,131.0,PreSubmitted


# To be continued ...
- Make next period (week: NIFTY, month: for rest) naked options
- Compute overbought and oversold from history to test df_nakeds

# FnOs

## Initialize `NSEfnos` class

In [ ]:
nse = NSEfnos()

## Get list of fnos

In [ ]:
fnos = nse.indexes() | nse.equities()

## Get a random fno quote

In [ ]:
import random

symbol = random.choice(list(fnos))
# symbol = 'NIFTY'

# Build `nakeds`

## Get PUT and CALL option prices using black scholes on option's iv

In [ ]:
df_nakeds = make_earliest_naked_opts(symbol)

df_nakeds

## Get All chains

In [ ]:
stock_chain = nse.equities_option_chain(symbol)  # Equity option chains
data = stock_chain.get("records").get("data")

# idx_chain = nse.index_option_chain("NIFTY")  # Index Option chains
# data = idx_chain.get('records').get('data')

pe = [data[i].get("PE") for i in range(len(data))]
df_pe = json_normalize(pe, sep="-").dropna(subset=["identifier"])
# df_pe = df_pe.sort_values('strikePrice', ascending=False).reset_index(drop=True)
df_pe["right"] = "P"

ce = [data[i].get("CE") for i in range(len(data))]
df_ce = json_normalize(ce, sep="-").dropna(subset=["identifier"])
df_ce["right"] = "C"
# df_ce = df_ce.sort_values('strikePrice', ascending=False).reset_index(drop=True)

df = pd.concat([df_ce, df_pe], ignore_index=True)

chain_col_map = {
    "underlying": "symbol",
    "underlyingValue": "undPrice",
    "expiryDate": "expiry",
    "strikePrice": "strike",
    "right": "right",
    "openInterest": "oi",
    "changeinOpenInterest": "oi_chg",
    "pchangeinOpenInterest": "oi_pchg",
    "totalTradedVolume": "volume",
    "change": "change",
    "pChange": "ltp_pct_chg",
    "totalBuyQuantity": "buy_qty",
    "totalSellQuantity": "sell_qty",
    "bidQty": "bid_qty",
    "bidprice": "bid",
    "askQty": "ask_qty",
    "askPrice": "ask",
    "impliedVolatility": "iv",
    "dte": "dte",
    "stdev": "stdev",
    "sigma": "sigma",
    "safe_strike": "safe_strike",
    "lastPrice": "ltp",
    "xPrice": "xPrice",
    "lot": "lot",
}

df.rename(columns=chain_col_map, inplace=True, errors="ignore")

# Convert expiry to UTC NSE eod
df = df.assign(expiry=df.expiry.apply(lambda x: convert_to_utc_datetime(x, eod=True)))

# Convert the rest to numeric
df = df.apply(convert_to_numeric)

# Replace zero values of iv and ltp to np.nan
df["iv"] = df["iv"].replace(0, np.nan)
df["ltp"] = df["ltp"].replace(0, np.nan)

# Gets dte
df["dte"] = get_dte(df.expiry)

# Sort the columns
df = df.sort_values(
    ["dte", "right", "strike"], ascending=[True, True, False]
).reset_index(drop=True)

# Fill missing ivs
df.iv = fbfillnas(df.iv)

# Put the lots
df = df.assign(lot=lot)

# Put the stdev

# Prettify the columns
cols = pretty_columns(df, chain_col_map)
df = df[cols]

In [ ]:
df

In [ ]:
mask = (df.undPrice > df.safe_strike) & (df.right == "C") * (df.dte < 9)
df[mask].head()

In [ ]:
df.loc[(df.safe_strike / df.undPrice).sort_values().index]

In [ ]:
df.assign(rom=df.ltp / df.undPrice * 14 / 100 * df.lot * 365 / df.dte)

In [ ]:
100842 / 1806.9 / 400

In [ ]:
undPrice = 2820
strike = 3080
right = "C"

In [ ]:
margin = 219780
dte = 43.048046
price = 112.45
price * lot / margin * 365 / dte

In [ ]:
6855 / 228380 * 365 / 8

In [ ]:
882.55 - 68

In [ ]:
# Split df calls / puts with strikes above / below undPrice
c_mask = (df.right == "C") & (df.strike > df.undPrice)
dfc = df[c_mask]

p_mask = (df.right == "P") & (df.strike <= df.undPrice)
dfp = df[p_mask]

In [ ]:
# Risk-free rate
r = 0.065  # 6.5%

# Calculate option prices using Black-Scholes model
df["xPrice"] = df.apply(
    lambda row: black_scholes(
        S=row["undPrice"],
        K=row["strike"],
        T=row["dte"] / 365,  # Convert days to years
        r=r,
        sigma=row["iv"] / 100,  # Convert percentage to decimal
        option_type=row["right"],
    ),
    axis=1,
)

# Round xPrice to 2 decimal places for readability
df["xPrice"] = df["xPrice"].round(2)

df

In [ ]:
cols = ["symbol", "undPrice", "strike", "dte", "right", "iv"]
pd.concat((dfc[cols].sample(3), dfp[cols].sample(3)), ignore_index=True).to_dict(
    "records"
)

In [ ]:
dfc = dfc.assign(xPrice=get_call_prices(dfc))
dfp = dfp.assign(xPrice=get_put_prices(dfp))

In [ ]:
risk_free_rate

In [ ]:
# dfp[~dfp.xPrice.isnull()]
dfp[chain_cols].head(1)

In [ ]:
call_price = black_scholes_call(
    spot_price, strike_price, time_to_expiry, risk_free_rate, implied_volatility
)
put_price = (
    call_price - spot_price + strike_price * np.exp(-risk_free_rate * time_to_expiry)
)

In [ ]:
black_scholes_put(882, 880, 8.141136 / 365, risk_free_rate * PUTSTDMULT, 29.66)

In [ ]:
# Get expected prices from black-scholes
df = pd.concat((dfc, dfp), ignore_index=True)

# Choose columns to show
chain_cols = [v for _, v in chain_col_map.items() if v in df.columns]
df = df[chain_cols]
df

In [ ]:
# Finding the closest above CALLSTDMULT rows
# ... compute the maximum stdev for the dte
std_max = max((dfc.strike - dfc.undPrice) / dfc.stdev)
std_max = max(std_max, CALLSTDMULT)  # ensure it meets the STDMULT threshold

In [ ]:
from scipy.stats import norm


def black_scholes_call(
    spot_price, strike_price, time_to_expiry, risk_free_rate, implied_volatility
):
    """
    This function calculates the theoretical value of a European call option using the Black-Scholes model.

    Args:
        spot_price (float): The current price of the underlying asset.
        strike_price (float): The strike price of the option.
        time_to_expiry (float): The time to expiration of the option in years.
        risk_free_rate (float): The risk-free interest rate in percentage.
        implied_volatility (float): The implied volatility of the underlying asset in percentage.

    Returns:
        float: The theoretical value of the call option.
    """

    # Convert implied volatility and risk-free rate from percentages to decimals
    implied_volatility /= 100
    risk_free_rate /= 100

    # Calculate d1 and d2
    d1 = (
        math.log(spot_price / strike_price)
        + (risk_free_rate + 0.5 * implied_volatility**2) * time_to_expiry
    ) / (implied_volatility * math.sqrt(time_to_expiry))
    d2 = d1 - implied_volatility * math.sqrt(time_to_expiry)

    # Calculate the call option value
    call_value = spot_price * norm.cdf(d1) - strike_price * math.exp(
        -risk_free_rate * time_to_expiry
    ) * norm.cdf(d2)

    return call_value

In [ ]:
dfc

In [ ]:
def get_closest_values(myArr: list, myNumber: float, how_many: int = 0):
    """Get closest values in a list

    how_many: 0 gives the closest value\n
              1 | 2 | ... use for CALL fences\n
             -1 | -2 | ... use for PUT fences"""

    i = 0

    result = dict()

    while i <= abs(how_many):

        if how_many >= 0:
            # going right
            val = myArr[myArr > myNumber].min()
            idx = np.where(myArr[myArr > myNumber] == val)[0][0]
            # print({'val': val, 'idx': idx})

        elif how_many <= 0:
            # going left
            val = myArr[myArr < myNumber].max()
            idx = np.where(myArr[myArr < myNumber] == val)[0][0]
            # print({'val': val, 'idx': idx})

        # else:
        #     val = min(myArr, key=lambda x: abs(x-myNumber))

        # idx = np.where(myArr == val)[0][0]

        result[val] = idx
        myNumber = val
        i += 1

    # output = list(result)[0:abs(1 if how_many == 0 else abs(how_many))]
    output = result

    return output

In [ ]:
def find_indices(arr, my_num, how_many):
    # Convert arr to numpy array if it's not already
    arr = np.array(arr)

    if how_many > 0:
        # Find indices of numbers greater than or equal to my_num
        indices = np.where(arr >= my_num)[0]
        # Sort indices based on the difference from my_num (ascending order)
        indices = indices[np.argsort(np.abs(arr[indices] - my_num))]
        return indices[: min(how_many, len(indices))]

    elif how_many < 0:
        # Find indices of numbers less than or equal to my_num
        indices = np.where(arr <= my_num)[0]
        # Sort indices based on the difference from my_num (ascending order)
        indices = indices[np.argsort(np.abs(arr[indices] - my_num))]
        return indices[: min(abs(how_many), len(indices))]

    else:  # how_many == 0
        # Find index of the closest value to my_num
        return np.argmin(np.abs(arr - my_num))

In [ ]:
mask = (df.right == "C") & (df.strike > df.undPrice)

dfc = df[mask].sort_values(["dte", "strike"], ascending=[True, False])
df1 = dfc[dfc.dte == dfc.dte.min()][chain_cols]
(df1.strike / (df1.undPrice + df1.stdev)).apply(get_prob)

In [ ]:
get_prob(1.5)

In [ ]:
closest_devs = dfc.groupby("dte").stdev.apply(
    lambda x: find_indices(x, CALLSTDMULT, 2)[0]
)
closest_devs

In [ ]:
CALLSTDMULT

In [ ]:
myArr = dfc.stdev
myNumber = CALLSTDMULT
how_many = 1

val = myArr[myArr > myNumber].min()
idx = np.where(myArr == val)[0][0]

In [ ]:
dfcs = dfc[dfc.stdev.isin(closest_devs.to_list())]

In [ ]:
strike = list(range(100, 150, 5))[1:] * 2

In [ ]:
dte = [5, 10] * int(len(strike) / 2)

In [ ]:
size = len(strike)
random_numbers = np.random.rand(size)

nan_probability = 0.3
nan_mask = np.random.random(size) < nan_probability
iv = np.where(nan_mask, np.nan, random_numbers)

p_choice = 0.5
c_choice = 1 - p_choice

right = np.random.choice(["P", "C"], size, p=[p_choice, c_choice])

marker = 0.3

In [ ]:
df = pd.DataFrame({"strike": strike, "dte": dte, "iv": iv, "right": right})

In [ ]:
def choose_closest_iv(df, marker):
    """Chooses the closest iv value to marker for each group in df.

    Args:
        df (pd.DataFrame): Dataframe containing columns 'strike', 'dte', 'iv', and 'right'.
        marker (float): The value to which the closest iv should be chosen.

    Returns:
        pd.DataFrame: A new DataFrame with the same columns as the input df,
                      but with the 'iv' column replaced by the closest values.
    """

    def g(group):
        """Function applied to each group in df.

        Args:
            group (pd.DataFrame): A subgroup of df based on 'dte' and 'right'.

        Returns:
            pd.DataFrame: The group with the 'iv' column replaced by the closest values.
        """
        if group["right"].iloc[0] == "P":
            # For Put options, choose the closest value below marker
            return (
                group[group["iv"] <= marker]
                .sort_values(by="iv", ascending=False)
                .head(1)
            )
        else:
            # For Call options, choose the closest value above marker
            return (
                group[group["iv"] >= marker]
                .sort_values(by="iv", ascending=True)
                .head(1)
            )

    return df.groupby(["dte", "right"]).apply(g)

In [ ]:
choose_closest_iv(df.copy(), marker)

In [ ]:
idx_chain = nse.index_option_chain("NIFTY")  # Index Option chains
curr_option_chain = nse.currency_option_chain("USDINR")  # Currency option chains

# Stock History

## Get Stock History

In [ ]:
df = nse.stock_history(symbol, days=365, chunks=50)

df.head()

# Index History
 - Index histories are treated in a separate class as:
     - its URL (https://niftyindices.com) is different than equity
     - which needs a POST after of a GET request
     - and also has its date string with a dash (20-Jul-2024)

## Initialize Index Histories

In [ ]:
idx = IDXHistories()

df_idx_hist = idx.make_histories(20)  # give number of days needed

df_idx_hist.groupby("symbol").head(3)

# Miscellaneous

## Stock Price (Stock Only! NOT FOR INDEX)

In [ ]:
symbol = "RELIANCE"
q = nse.stock_quote(symbol)

stock_price = q.get("priceInfo").get("lastPrice")

stock_price